In [1]:
import os
import torch 
import numpy as np
import pandas as pd
import torch.nn as nn
from datetime import datetime
from scipy.integrate import odeint
import matplotlib.pyplot as plt
#from model import SuEIR_Corr, weight_fun
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3,4,5,6,7"
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

In [8]:
direc = "/home/ec2-user/SageMaker/efs/Danielle/Ray/Data/US_Reports/"
list_csv = sorted(os.listdir(direc))[1:]
us = []
for file in list_csv:
    sample = pd.read_csv(direc + file).set_index("Province_State")[["Confirmed", "Recovered", "Deaths"]].sort_values(by = "Confirmed", ascending = False)
    us.append(sample.drop(['Diamond Princess', 'Grand Princess']))
us = pd.concat(us, axis=1, join='inner')[:55] 
us_data = us.values.reshape(55,-1,4)
us_data[us_data!=us_data] = 0
us_data_diff = np.diff(us_data, axis = 1)
us_data = us_data[:,1:]

# us_data[:,:,1] += us_data[:,:,2]
# win = 3
# for i in range(win, len(us_data_diff)):
#     us_data_diff[:,i] = np.mean(us_data_diff[:,i-win:i], axis = 1)

# # standardization
# std = np.std(us_data_diff, axis = (1), keepdims = True)
# avgs = np.mean(us_data_diff, axis = (1), keepdims = True)
# us_data_diff = (us_data_diff - avgs)/std
# us_data_diff[us_data_diff!=us_data_diff] = 0

In [ ]:
# first day 04/12
# list_csv[65] list_csv[72] list_csv[79]
# list_csv[84] list_csv[91] list_csv[98]

In [ ]:
train_test_split = 84
train = us_data[:,:train_test_split-7]

In [ ]:
k = 0
for j in range(14, train.shape[1]-7):
    for i in range(train.shape[0]):
        sample = torch.from_numpy(train[i,j-14:j+7]).float()
        torch.save(sample, "/home/ec2-user/SageMaker/efs/Danielle/Ray/Data/Latest_US_Covid_Data_0726/train/sample_" + str(k) + ".pt")
        k += 1

In [ ]:
k = 0
for w in [0,7,14]:
    test = us_data[:,(train_test_split + w - 14):(train_test_split + w + 7)]
    for i in range(test.shape[0]):
        sample = torch.from_numpy(test[i]).float()
        torch.save(sample, "/home/ec2-user/SageMaker/efs/Danielle/Ray/Data/Latest_US_Covid_Data_0726/test/sample_" + str(k) + ".pt")
        k += 1